# Segmenting and Clustering Toronto City Data

## Part 1

Toronto neighborhood data will be scraped from Wikipedia and compiled into a pandas dataframe. Using FourSquare's api, the neighborhoods will be explored and data collected about local venues. This local data will be used to characterized each individual neighborhood. KMeans clustering will be used to segregate the neighborhoods into groups with similar characteristics. 

In [1]:
#needed libraries
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

%matplotlib inline

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url, 'lxml')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [3]:
table = soup.find('table') #locate the table with our data from the wikipedia page
headers = []
for i in table.find_all('th'):
    for head in i:
        headers.append(head)
        
print(headers)

['Postcode', 'Borough', 'Neighbourhood\n']


In [4]:
headers[-1] = headers[-1].strip()
headers

['Postcode', 'Borough', 'Neighbourhood']

In [5]:
df=pd.DataFrame(columns = headers, dtype='str')
df

,Postcode,Borough,Neighbourhood


In [6]:
my_list = []
for row in table.find_all('tr'):
        columns = row.find_all('td')
        
        for column in columns:
            my_list.append(column.get_text())           


In [7]:
A=[]
B=[]
C=[]

for i in range(0,289,3):
    A.append(my_list[i])
    print(len(A))  #want to make sure same number of values
    
for i in range(1,290,3):
    B.append(my_list[i])
    print(len(B))
    
for i in range(2,291,3):
    C.append(my_list[i])
    print(len(C))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [8]:
#assign variables to dataframe
df['Postcode'] = A
df['Borough'] = B
df['Neighbourhood'] = C

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [9]:
#remove the newline character (\n) from each item
#in the Neighbourhood column
for col in df:
    if col == 'Neighbourhood':
        for i, n in df[col].iteritems():
            df[col][i] = n.strip()
            df.at[i, 'Neighbourhood'] = df[col][i]
            

In [10]:
df['Borough']=df['Borough'].replace('Not assigned', np.NaN) 
df=df[pd.notnull(df['Borough'])] #removes rows with not assigned boroughs
df.head()


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [11]:
df=df.reset_index(drop=True) #reset index to compensate for dropped rows


In [12]:
#combine Neighborhoods sharing postcode and borough
#into comma separated list
df=df.groupby('Postcode')['Borough', 'Neighbourhood'].agg(lambda col: ', '.join(col)).reset_index() 

In [13]:
#remove the duplicates from the Borough column
for i in range(0, len(df['Borough'])):
    string = df['Borough'][i].split(',')
    df['Borough'][i] = string[0]

In [14]:
for n in range(0, len(df['Neighbourhood'])):
    if df['Neighbourhood'][n] == 'Not assigned':       #naming any not assigned
        df['Neighbourhood'][n] = df['Borough'][n]     # neighbourhoods
    else:
        continue
        
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M2H,North York,Hillcrest Village
8,M2J,North York,"Fairview, Henry Farm, Oriole"
9,M2K,North York,Bayview Village


In [15]:
df.shape

(41, 3)

## Part 2
Now that the data has been wrangled and cleaned we need to add location data to communicate with
the FourSquare api. 

In [16]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
coordinates.shape

(103, 3)

In [18]:
coordinates.rename(columns = {'Postal Code' : 'Postcode'}, inplace = True)

In [19]:
t_df = pd.merge(df, coordinates, on ='Postcode', how = 'left')
t_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Now we create the relevant maps and call on FourSquare to explore these neighborhoods.

In [20]:
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = 'can_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("Toronto coordinates: {}, {}.".format(latitude, longitude))

Toronto coordinates: 43.653963, -79.387207.


In [22]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start = 11) 

#mark each neighborhood on the map
for lat, lng, borough, neighborhood in zip(t_df['Latitude'], t_df['Longitude'], t_df['Borough'], t_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3189cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [23]:
toronto_df = t_df[t_df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
23,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
24,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
25,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
26,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [24]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

### Explore one neighborhood and then scale that search to all of them

In [25]:
toronto_df.reset_index(inplace = True, drop = True)

In [26]:
#location data for first neighborhood in df
neighborhood_latitude = toronto_df.loc[0,'Latitude']
neighborhood_longitude = toronto_df.loc[0,'Longitude']
neighborhood_name = toronto_df.loc[0, 'Neighbourhood']

print('Location of {} is {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))


Location of The Beaches is 43.67635739999999, -79.2930312.


### Send the GET request to FourSquare

In [27]:
#FourSquare credentials
CLIENT_ID = 'P5WHJAKGQUMOU1XSLZMM2SGSIIDMKRIMZLHGFVE3AJX5GC42' 
CLIENT_SECRET = 'S21DNPMZCAEDPPZ1OTAJIRPYTURWDWX3IBGE45DJEQADRXPP' 
VERSION = '20190301'
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9f9543db04f53b168ba0a8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [29]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

In [30]:
filtered_col = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_col]

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
1,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
2,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.678798,-79.298045
3,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869
4,Dip 'n Sip,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.678897,-79.297745


In [33]:
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis = 1)
nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


The above is the top venues near our one search. Let's scale this up to all the neighborhoods in Toronto
to find top venues in each of them. 

The below is a function designed to retrieve the top 100 venues for each neighborhood.

In [34]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
                            'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
        
    return(nearby_venues)

In [35]:
toronto_venues = get_nearby_venues(names= toronto_df['Neighbourhood'],
                                  latitudes = toronto_df['Latitude'],
                                  longitudes = toronto_df['Longitude'])

The Beaches
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity


In [36]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
5,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
6,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
7,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
8,"Harbourfront, Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
9,"Harbourfront, Regent Park",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [37]:
toronto_venues.groupby('Neighborhood').count() 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
Central Bay Street,83,83,83,83,83,83
Christie,16,16,16,16,16,16
"Dovercourt Village, Dufferin",18,18,18,18,18,18
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Harbourfront, Regent Park",46,46,46,46,46,46
"Little Portugal, Trinity",65,65,65,65,65,65
"Ryerson, Garden District",100,100,100,100,100,100


In [38]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Find the Frequency of each venue occurrency in each neighborhood

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.030000,0.0,0.0,0.010000,0.010000,0.03,0.0000,...,0.000000,0.040000,0.01,0.0,0.0,0.010000,0.0,0.0,0.010000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.0,0.0,0.018182,0.000000,0.00,0.0000,...,0.018182,0.018182,0.00,0.0,0.0,0.018182,0.0,0.0,0.000000,0.00
2,Central Bay Street,0.012048,0.00,0.012048,0.0,0.0,0.000000,0.012048,0.00,0.0000,...,0.012048,0.024096,0.00,0.0,0.0,0.012048,0.0,0.0,0.012048,0.00
3,Christie,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0625,...,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00
4,"Dovercourt Village, Dufferin",0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0000,...,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00


### Find the top 10 most common venues for a given neighborhood and cluster them for analysis. 

In [40]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

In [41]:
num_top_venues = 10
labels=['st', 'nd', 'rd']
columns = ['Neighborhood']

for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, labels[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
        
#new df for our sorted venues
sorted_venues = pd.DataFrame(columns=columns)
sorted_venues['Neighborhood'] = toronto_grouped['Neighborhood']

In [42]:
for i in np.arange(toronto_grouped.shape[0]):
    sorted_venues.iloc[i,1:] = most_common_venues(toronto_grouped.iloc[i,:], num_top_venues)

sorted_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Burger Joint,Asian Restaurant,Bakery,Hotel,Sushi Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Steakhouse,Restaurant,Pub,Café,Cheese Shop
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Café,Bar,Burger Joint,Salad Place,Thai Restaurant,Chinese Restaurant
3,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Nightclub,Athletics & Sports,Diner,Baby Store,Italian Restaurant
4,"Dovercourt Village, Dufferin",Pharmacy,Supermarket,Discount Store,Bakery,Park,Brewery,Gym / Fitness Center,Pool,Café,Liquor Store


In [43]:
# number of clusters
kclusters = 5

toronto_clusters = toronto_grouped.drop('Neighborhood', 1)
kmeans= KMeans(n_clusters=kclusters, random_state=3).fit(toronto_clusters)

#examine the labels generated for each row
kmeans.labels_[0:11]

array([1, 1, 1, 4, 3, 1, 1, 0, 1, 1, 2], dtype=int32)

In [44]:
sorted_venues.insert(0, 'Cluster Labels', kmeans.labels_) #add cluster labels

toronto_labeled = toronto_df

toronto_labeled = toronto_labeled.join(sorted_venues.set_index('Neighborhood'), on='Neighbourhood')

toronto_labeled.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Pub,Health Food Store,Grocery Store,Cupcake Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
1,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Dessert Shop
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Pizza Place,Bar,Restaurant,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Cosmetics Shop,Italian Restaurant,Gastropub,Bakery
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Steakhouse,Restaurant,Pub,Café,Cheese Shop


Map the clusters for visualization

In [45]:
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=12)

import matplotlib.cm as cm
import matplotlib.colors as colors

# color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
marker_color=[]
for lat, lng, hood, cluster in zip(toronto_labeled['Latitude'], toronto_labeled['Longitude'],
    toronto_labeled['Neighbourhood'], toronto_labeled["Cluster Labels"]):
            label=folium.Popup(str(hood) + ': Cluster '+ str(cluster), parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius = 5,
                popup=label,
                color=rainbow[cluster-1],
                fill_opacity=0.7).add_to(cluster_map)
    
cluster_map
                                               

## Cluster 1


In [46]:
toronto_labeled.loc[toronto_labeled['Cluster Labels'] == 0, toronto_labeled.columns[[1]+[2] + list(range(5,toronto_labeled.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,West Toronto,"Little Portugal, Trinity",0,Bar,Coffee Shop,Asian Restaurant,Boutique,Restaurant,Pizza Place,Café,New American Restaurant,Cocktail Bar,Men's Store


## Cluster 2

In [47]:
toronto_labeled.loc[toronto_labeled['Cluster Labels'] == 1, toronto_labeled.columns[[1]+[2] + list(range(5,toronto_labeled.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,"Harbourfront, Regent Park",1,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Dessert Shop
2,Downtown Toronto,"Ryerson, Garden District",1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Pizza Place,Bar,Restaurant,Italian Restaurant
3,Downtown Toronto,St. James Town,1,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Cosmetics Shop,Italian Restaurant,Gastropub,Bakery
4,Downtown Toronto,Berczy Park,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Steakhouse,Restaurant,Pub,Café,Cheese Shop
5,Downtown Toronto,Central Bay Street,1,Coffee Shop,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Café,Bar,Burger Joint,Salad Place,Thai Restaurant,Chinese Restaurant
6,Downtown Toronto,"Adelaide, King, Richmond",1,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Burger Joint,Asian Restaurant,Bakery,Hotel,Sushi Restaurant
7,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Pizza Place,Fried Chicken Joint,Brewery,Scenic Lookout,Baseball Stadium


## Cluster 3

In [48]:
toronto_labeled.loc[toronto_labeled['Cluster Labels'] == 2, toronto_labeled.columns[[1]+[2] + list(range(5,toronto_labeled.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,2,Coffee Shop,Pub,Health Food Store,Grocery Store,Cupcake Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store


## Cluster 4

In [49]:
toronto_labeled.loc[toronto_labeled['Cluster Labels'] == 3, toronto_labeled.columns[[1]+[2] + list(range(5,toronto_labeled.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,"Dovercourt Village, Dufferin",3,Pharmacy,Supermarket,Discount Store,Bakery,Park,Brewery,Gym / Fitness Center,Pool,Café,Liquor Store


## Cluster 5

In [50]:
toronto_labeled.loc[toronto_labeled['Cluster Labels'] == 4, toronto_labeled.columns[[1]+[2] + list(range(5,toronto_labeled.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,Christie,4,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Nightclub,Athletics & Sports,Diner,Baby Store,Italian Restaurant
